In [ ]:
!pip install transformers datasets trl wandb weave -U

In [18]:
from datasets import load_dataset
from trl import GRPOConfig, GRPOTrainer

In [19]:
dataset = load_dataset("parquet", data_files="data/rl_df.parquet", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [20]:
dataset[1100]

{'prompt': "Generate a funny joke related to this headline: 'Electricity costs jolt New Jersey's race for governor — and preview next year's midterms' by either modifying it or responding to it."}

In [21]:
# Dummy reward function for demonstration purposes
import random

def random_reward(completions, **kwargs):
    return [random.randint(0, 10) for c in completions]

In [22]:
training_args = GRPOConfig(output_dir="Qwen3-VL-2B-Instruct-GRPO", report_to="wandb")
trainer = GRPOTrainer(
    model="Qwen/Qwen3-VL-2B-Instruct",
    reward_funcs=random_reward,
    args=training_args,
    train_dataset=dataset,
)

In [ ]:
import weave
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 151645, 'bos_token_id': None, 'pad_token_id': 151643}.
`generation_config` default values have been modified to match model-specific defaults: {'temperature': 0.7, 'top_p': 0.8}. If this is not desired, please set these values explicitly.


Step,Training Loss
10,0.015800
20,-0.011700
30,0.013900
40,0.030500
50,0.024900
60,-0.030200
70,-0.010800
